In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from zipfile import ZipFile

In [ ]:
file_name = r'/content/gdrive/MyDrive/archive.zip'

In [ ]:
# with ZipFile(file_name, 'r') as zip:
#     zip.extractall()

In [ ]:
dir = r'/content/COVID-19_Radiography_Dataset'

In [ ]:
import os
import requests

In [ ]:
for folder in os.listdir(dir):
    if os.path.isdir(os.path.join(dir, folder)):
        for subfolder in os.listdir(os.path.join(dir, folder)):
            print(folder, subfolder, len(os.listdir(os.path.join(dir, folder, subfolder))))

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import tensorflow as tf

In [ ]:
for folder in os.listdir(dir):
    if os.path.isdir(os.path.join(dir, folder)):
        for subfolder in os.listdir(os.path.join(dir, folder)):
          if subfolder == 'images':
            print(folder, subfolder)
            img = cv2.imread(os.path.join(dir, folder, subfolder, os.listdir(os.path.join(dir, folder, subfolder))[0]))
            print('image shape {}'.format(img.shape))
            plt.imshow(img)
            plt.show()

In [ ]:
def load_images(path, urls, target):
    images = []
    labels = []
    for i in range(len(urls)) :
        img_path = path + '/' + urls[i]
        img = cv2.imread(img_path)
        img = img / 255.0
        img = cv2.resize(img, (50, 50))
        images.append(img)
        labels.append(target)
    images = np.array(images)
    print(images.shape)
    return images, labels

In [ ]:
for folder in os.listdir(dir):
    if os.path.isdir(os.path.join(dir, folder)):
        if folder == 'COVID':
            for subfolder in os.listdir(os.path.join(dir, folder)):
                if subfolder == 'images':
                  covid_path = os.path.join(dir, folder, subfolder)
                  covid_urls = os.listdir(covid_path)
                  covid_images, covid_target = load_images(covid_path , covid_urls, 1)

In [ ]:
for folder in os.listdir(dir):
    if os.path.isdir(os.path.join(dir, folder)):
        if folder == 'Lung_Opacity':
            for subfolder in os.listdir(os.path.join(dir, folder)):
                if subfolder == 'images':
                  covid_path = os.path.join(dir, folder, subfolder)
                  covid_urls = os.listdir(covid_path)
                  mid_covid_images, mid_covid_target = load_images(covid_path , covid_urls, 2)

In [ ]:
for folder in os.listdir(dir):
    if os.path.isdir(os.path.join(dir, folder)):
        if folder == 'Normal':
            for subfolder in os.listdir(os.path.join(dir, folder)):
                if subfolder == 'images':
                  covid_path = os.path.join(dir, folder, subfolder)
                  covid_urls = os.listdir(covid_path)
                  no_covid_images, no_covid_target = load_images(covid_path , covid_urls, 0)

In [ ]:
for folder in os.listdir(dir):
    if os.path.isdir(os.path.join(dir, folder)):
        if folder == 'Viral Pneumonia':
            for subfolder in os.listdir(os.path.join(dir, folder)):
                if subfolder == 'images':
                  covid_path = os.path.join(dir, folder, subfolder)
                  covid_urls = os.listdir(covid_path)
                  low_covid_images, low_covid_target = load_images(covid_path , covid_urls, 3)

In [ ]:
low_covid_images.shape

In [ ]:
all_data = np.r_[covid_images, no_covid_images, mid_covid_images, low_covid_images]

In [ ]:
all_data.shape

In [ ]:
targets = np.r_[covid_target, no_covid_target, mid_covid_target, low_covid_target]
targets.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(all_data, targets, test_size=0.2)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
model = Sequential()
model.add(Conv2D(32, 3, input_shape=(50,50,3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, 3, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, 3, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, 3, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_test, y_test))

In [ ]:
def get_metrics(history):
    fig = plt.figure(figsize = (10,10))
    plt.subplot(2,2,1)
    plt.plot(history.history["accuracy"], label = "training accuracy")
    plt.plot(history.history["val_accuracy"], label = "test accuracy")
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")

    plt.subplot(2,2,2)
    plt.plot(history.history["loss"], label = "train loss")
    plt.plot(history.history["val_loss"], label = "test loss")
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Loss")

In [ ]:
history = model.history
get_metrics(history)

In [ ]:
model.save('covid_classification.h5')

In [ ]:
model_to_predict = tf.keras.models.load_model('covid_classification.h5')
def predict_covid(test_image):
  img = cv2.imread(test_image)
  img = img / 255.0
  img = cv2.resize(img, (50, 50))
  img = img.reshape(1,50,50,3)
  prediction = model_to_predict.predict(img)
  pred_class = np.argmax(prediction, axis = -1)
  return pred_class

In [ ]:
print(predict_covid(r'/content/COVID-19_Radiography_Dataset/COVID/images/COVID-1.png'))